# Analyzing the Titanic Data

## Introductionary words

This **project** is part of the [Udacity data analyst nanodegree](https://www.udacity.com/course/data-analyst-nanodegree--nd002).

This analysis is done by **[Guillaume Simler](https://github.com/guillaumesimler)** as part of his nanodegree's graduation.

For more infos, please have a look at the related [githup repo](https://github.com/guillaumesimler/nanodap1)